## Market Analysis - Restaurants in San Francisco

### Business Problem:
A friend is thinking about opening a restaurant in San Francisco, however, he's not sure about which Neighborhood might be the best place. Besides, he doesn't know what type of cuisine either, therefore, he'd like to know what type of restaurants are more frequent in different areas of the city.

### Data:
My idea is to use Foursquare's API to download different restaurants information about each Neighborhood, so I can easily see which restaurants are more popular in different areas of the city. Besides, I think it'd be great to know the population of each area, which is the 'potential market' for the restaurant.

- The problem with the API is that it's limited to 100 venues per call, therefore, our analysis could be compromised by the fact that we don't have access to all the data.

In [1]:
#Import Libraries
import pandas as pd
from uszipcode import SearchEngine
import requests 
from bs4 import BeautifulSoup 
import numpy as np
import re

In [2]:
#Import file with Neighborhoods and Zipcodes and use uszipcode to get the geolocation
search = SearchEngine(simple_zipcode=True)
sf_df = pd.read_csv('sf_neighborhoods.csv')

#Create a loop to go through all zip codes and get the Latitude and Longitude.
master_list = []
for zip_code in sf_df['Zip Code']:
    temp_lst = []
    temp_dic = search.by_zipcode(zip_code).to_dict()
    lat = temp_dic['lat']
    lng = temp_dic['lng']
    temp_lst.append(lat)
    temp_lst.append(lng)
    master_list.append(temp_lst)

#Create a data frame with that information and merge it with the Neighborhoods using the index.
coor_df = pd.DataFrame(data=master_list, columns=['lat','lng'])
sf_df = pd.merge(sf_df, coor_df, left_index=True, right_index=True)

### Foursquare API

In [3]:
#My User ID
CLIENT_ID = 'UOYNHZFOHY5ETVSAPG4NKSH20V3PPSFSZCCRLLT02BRLDUU1' # your Foursquare ID
CLIENT_SECRET = 'OTGN5Q4YMFVIQR3XCV3K4XLJ3H1HFYLIJZJALDVYRGGF3AWK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000
SECTION = 'food'

#Function to pull data and put in a good format
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            SECTION)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'])for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#Create sf_venues data frame with the data
sf_venues = getNearbyVenues(names=sf_df['Neighborhood'],latitudes=sf_df['lat'],longitudes=sf_df['lng'])

Hayes Valley/Tenderloin/North of Market 
South of Market 
Potrero Hill 
Chinatown 
Polk/Russian Hill (Nob Hill) 
Inner Mission/Bernal Heights 
Ingelside-Excelsior/Crocker-Amazon 
Castro/Noe Valley 
Western Addition/Japantown 
Parkside/Forest Hill 
Haight-Ashbury 
Inner Richmond 
Outer Richmond 
Sunset 
Marina 
Bayview-Hunters Point 
St. Francis Wood/Miraloma/West Portal 
Twin Peaks-Glen Park 
Lake Merced 
North Beach/Chinatown 
Visitacion Valley/Sunnydale 


In [4]:
sf_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Robin,37.779127,-122.423378,Sushi Restaurant
1,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Brenda's French Soul Food,37.782896,-122.418897,Southern / Soul Food Restaurant
2,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Saigon Sandwich,37.783084,-122.417650,Sandwich Place
3,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Monsieur Benjamin,37.777478,-122.423131,French Restaurant
4,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Nojo Ramen Tavern,37.776637,-122.421270,Ramen Restaurant


In [5]:
#Now I need to keep only venues that contain the word 'Restaurant'
sf_rest = sf_venues[sf_venues['Venue Category'].str.contains('Restaurant')]
sf_rest['Venue Category'] = sf_rest['Venue Category'].str.replace(' Restaurant','')

#Create another data frame grouping neighborhood and venue category so I can get a count of Category by Neighborhood
df_master = sf_rest.groupby(['Neighborhood','Venue Category']).count()['Venue'].reset_index()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### BeautifulSoup to scrape Population by Zip Code data from website
Next, I think it would be interesting to have an idea of the market size in each neighborhood, for that I'm going to scrape data from a website that contains Zip Codes and Population.

In [6]:
#Collect and parse
page = requests.get('https://www.california-demographics.com/zip_codes_by_population')
soup = BeautifulSoup(page.text, 'html.parser')

#Find Table with data
artist_name_list = soup.find(class_='ranklist span8')

#Extract what I need
artist_name_list_items = artist_name_list.find_all('td')

#Loop through each element and put them on a list
temp_list = []
for items in artist_name_list_items:
    temp_list.append(items.text.replace(' ','').replace('\n',''))

#Create Data Frame with list
zip_code = temp_list[1::3]
pop = temp_list[2::3]
df_zip = pd.DataFrame(data=zip_code,columns=['Zip Code'])
df_pop = pd.DataFrame(data=pop,columns=['Population'])
pop_df = pd.merge(df_zip, df_pop, left_index=True, right_index=True)

pop_df.head()

,Zip Code,Population
0,90011,"108,051"
1,90650,"106,404"
2,91331,"105,696"
3,90201,"102,878"
4,92335,"99,791"


### Some Zip Codes are combined into one row, so I need to deal with that

In [7]:
#Create a loop to duplicate the row for records with multiple Zip Codes
lst11 = []
for idx, code in enumerate(list(pop_df['Zip Code'])):
    for code2 in re.split(',|and',code):
        temp_list = []
        lst11.append([code2,pop_df['Population'][idx]])

pop_df = pd.DataFrame(lst11, columns=['Zip Code', 'Population'])
pop_df.head()

,Zip Code,Population
0,90011,"108,051"
1,90650,"106,404"
2,91331,"105,696"
3,90201,"102,878"
4,92335,"99,791"


### Create Data Frame to analyze data

In [8]:
#Merge df_master with sf_df to get Neighborhood name and then merge with pop_df to get Population information
df_analyze = pd.merge(df_master, sf_df[['Zip Code','Neighborhood']], on='Neighborhood', how='left')
df_analyze['Zip Code'] = df_analyze['Zip Code'].astype(str)
df_analyze = pd.merge(df_analyze, pop_df, on='Zip Code', how='left')

### Population/Venue: A ratio between total population by number of Venues in a zip code will give me an idea of the size of the market and with how many restaurants we'll be competing with. The higher the better, because then there's not a lot of restaurants in a given area for a population.

In [9]:
#Calculate total number of Venues by Neighborhood and put it in a column
total_venue = df_analyze.groupby('Neighborhood').sum()['Venue'].reset_index()
total_venue.columns = ['Neighborhood','Total Venue']

#Merge that with df_analyze and create a ratio of Population/Venue
df_analyze = pd.merge(df_analyze, total_venue, on='Neighborhood')
df_analyze['Population'] = df_analyze['Population'].str.replace(',','')
df_analyze['Population'] = pd.to_numeric(df_analyze['Population'])
df_analyze['Pop/Venue'] = round(df_analyze['Population']/df_analyze['Total Venue'],2)

df_analyze.head()

,Neighborhood,Venue Category,Venue,Zip Code,Population,Total Venue,Pop/Venue
0,Bayview-Hunters Point,African,1,94124,35492,37,959.24
1,Bayview-Hunters Point,American,3,94124,35492,37,959.24
2,Bayview-Hunters Point,Asian,1,94124,35492,37,959.24
3,Bayview-Hunters Point,Brazilian,1,94124,35492,37,959.24
4,Bayview-Hunters Point,Cantonese,1,94124,35492,37,959.24


### Category Analysis

In [12]:
df_analyze['Venue Category'].nunique()

61

#### There are 61 different types of cuisine in San Francisco!

### Top 10 most frequent categories

In [13]:
df_analyze.groupby('Venue Category').count()['Venue'].sort_values(ascending=False).head(10)

Venue Category
Mexican       20
Sushi         19
Italian       19
Japanese      19
Vietnamese    18
American      18
Thai          18
Korean        17
Indian        16
Seafood       16
Name: Venue, dtype: int64

That's really interesting, maybe this is a consequence of the diversity of the city, with lots of people from different countries, my hypothesis is that this environment create different needs for different types of cuisine, which translates into a wide variety of restaurants!

### Let's see the best Neighborhoods to open a restaurant based on the Population/Venue ratio.

In [10]:
df_analyze[['Neighborhood', 'Pop/Venue','Total Venue','Population']].drop_duplicates().sort_values('Pop/Venue', ascending=False)

,Neighborhood,Pop/Venue,Total Venue,Population
120,Ingelside-Excelsior/Crocker-Amazon,1641.79,52,85373
141,Inner Mission/Bernal Heights,1249.78,59,73737
375,Sunset,992.32,63,62516
0,Bayview-Hunters Point,959.24,37,35492
427,Visitacion Valley/Sunnydale,879.06,49,43074
277,Polk/Russian Hill (Nob Hill),870.57,65,56587
254,Parkside/Forest Hill,745.44,64,47708
70,Haight-Ashbury,726.83,60,43610
166,Inner Richmond,668.02,62,41417
190,Lake Merced,599.13,52,31155


#### Good Places

- Ingelside-Excelsior/Crocker-Amazon and Inner Mission/Bernal Heights: Look like good places to open a restaurant, however, I'm getting this conclusion because those are well populated areas.

- Bayview-Hunters Point and Visitacion Valley/Sunnydale: Not many restaurants relative to other neighborhoods.

#### Competitive Neighborhoods

- Chinatown, South of Market and Marina: Those places have a low ratio of Population per restaurant and both have a lot of open restaurants. Opening a new one in either of those neighborhoods could be challenging.

### Next, let's see the areas with more restaurants by cuisine type

In [11]:
df_analyze[df_analyze['Venue'] > 7]

,Neighborhood,Venue Category,Venue,Zip Code,Population,Total Venue,Pop/Venue
5,Bayview-Hunters Point,Chinese,8,94124,35492,37,959.24
116,Hayes Valley/Tenderloin/North of Market,Sushi,8,94102,30140,62,486.13
133,Ingelside-Excelsior/Crocker-Amazon,Mexican,8,94112,85373,52,1641.79
153,Inner Mission/Bernal Heights,Mexican,11,94110,73737,59,1249.78
193,Lake Merced,Chinese,9,94132,31155,52,599.13
219,Marina,Italian,9,94123,25461,60,424.35
241,North Beach/Chinatown,Italian,9,94133,26942,60,449.03
257,Parkside/Forest Hill,Chinese,23,94116,47708,64,745.44
286,Polk/Russian Hill (Nob Hill),Italian,8,94109,56587,65,870.57
299,Polk/Russian Hill (Nob Hill),Sushi,8,94109,56587,65,870.57


#### Highlights:

- Mexican Restaurants are mostly located in Inner Mission/Bernal Heights. Opening a new Mexican place would be really challenging over there.
- Parkside/Forest Hill look like a good place for Chinese food, however, not a good place to open a new one.

#### My Recommendation:
- I'd recommend choosing neighborhoods with less restaurants to reduce the number of competitors: Bayview-Hunters Point and Visitacion Valley/Sunnydale.


### Race Information
Next, I'm going to figure out the rate distribution in each area, as I was trying to find an API or something I could use to pull the data, I found the census data from 2010. Although it's not 100% accurate, I'm going to use it to define which type of restaurant would be the best in each area.

The data was manually downloaded and it will be imported to help me with my analysis.

####  Bayview-Hunters Point

In [45]:
#Import data
race_df = pd.read_csv('race.csv')

#Figure out distribution of race by zip code
race_total = race_df.groupby(['Zip Code', 'Race']).agg({'Population': 'sum'})
race_total = race_total.groupby(['Zip Code']).apply(lambda x:100 * x / float(x.sum()))
race_total = race_total.sort_values(['Zip Code','Population'], ascending=False).reset_index()

#Top 5 Races in 94124:
race_total[race_total['Zip Code'] == 94124].head()

,Zip Code,Race,Population
15,94124,Black or African American,35.522481
16,94124,Asian - Chinese,23.296124
17,94124,Some Other Race,15.894574
18,94124,White,12.719380
19,94124,Asian - Filipino,3.231008


In [46]:
#Current restaurants in the area
df_analyze[df_analyze['Neighborhood'] == 'Bayview-Hunters Point '].sort_values('Venue')

,Neighborhood,Venue Category,Venue,Zip Code,Population,Total Venue,Pop/Venue
0,Bayview-Hunters Point,African,1,94124,35492,37,959.24
2,Bayview-Hunters Point,Asian,1,94124,35492,37,959.24
3,Bayview-Hunters Point,Brazilian,1,94124,35492,37,959.24
4,Bayview-Hunters Point,Cantonese,1,94124,35492,37,959.24
7,Bayview-Hunters Point,Hawaiian,1,94124,35492,37,959.24
10,Bayview-Hunters Point,Restaurant,1,94124,35492,37,959.24
8,Bayview-Hunters Point,Latin American,2,94124,35492,37,959.24
1,Bayview-Hunters Point,American,3,94124,35492,37,959.24
6,Bayview-Hunters Point,Dim Sum,3,94124,35492,37,959.24
11,Bayview-Hunters Point,Southern / Soul Food,4,94124,35492,37,959.24


#### Restaurant type in Bayview-Hunters Point:
- The recommended cuisine in this area would be African cuisine, given that Black or African American represents 35% of the area and I see only 4 Southern/Sould Food restaurants. 
- Filipino cuisine could be also a possibility, despite the Filipino population not being really big in the area, I don't see any Filipino restaurants in the area.

In [47]:
race_total[race_total['Zip Code'] == 94134].head()

,Zip Code,Race,Population
0,94134,Asian - Chinese,41.122952
1,94134,White,16.918398
2,94134,Some Other Race,12.167446
3,94134,Asian - Filipino,10.386159
4,94134,Black or African American,9.974699


In [49]:
df_analyze[df_analyze['Neighborhood'] == 'Visitacion Valley/Sunnydale '].sort_values('Venue', ascending=False)

,Neighborhood,Venue Category,Venue,Zip Code,Population,Total Venue,Pop/Venue
441,Visitacion Valley/Sunnydale,Mexican,9,94134,43074,49,879.06
450,Visitacion Valley/Sunnydale,Vietnamese,5,94134,43074,49,879.06
447,Visitacion Valley/Sunnydale,Southern / Soul Food,4,94134,43074,49,879.06
440,Visitacion Valley/Sunnydale,Latin American,4,94134,43074,49,879.06
429,Visitacion Valley/Sunnydale,Asian,3,94134,43074,49,879.06
442,Visitacion Valley/Sunnydale,New American,2,94134,43074,49,879.06
448,Visitacion Valley/Sunnydale,Sushi,2,94134,43074,49,879.06
435,Visitacion Valley/Sunnydale,Hunan,2,94134,43074,49,879.06
437,Visitacion Valley/Sunnydale,Italian,2,94134,43074,49,879.06
428,Visitacion Valley/Sunnydale,American,2,94134,43074,49,879.06


#### Restaurant type in Visitacion Valley/Sunnydale:
- A Chinese restaurant could be a good idea in the area, if you look at the population data frame, Chinese represents 41% of the population while we have only one Chinese restaurant in the area.
- Filipino could also be a possiblity here, given the size of the population and the fact that there's only one Filipino restaurant in the area.

### Conclusion:
More data would be necessary to make a precise recommendation, it would be also interesting to see the rent price in each area, so we could estimate how expensive it would be to have a restaurant in each area. Another point is that we don't have all the data, the API limits to 100 venues per call, therefore, it's possible that the data is not complete.

Also, the population data is based off people who resides in the area, therefore, it's possible that during the day the distribution is different. Downtown areas will be busier during the week and it's important to know exactly what type of restaurant we are trying to create. 

Regardless, I think that this is already good for my friend, he will be happy with my suggestions.